# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
import csv



spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

#### Scope:
This project will pull data from all sources and create fact and dimension tables to show movement of other contries's person into US port city and the information of the population (sum, male, female) in this city.

#### data source:

- us-cities-demographics: comes from OpenSoft. It contains information about the demographics of all US cities and census-designated places with a population
- sas_data: comes from the US National Tourism and Trade Office, it has immigrants and the ports city which they arrive.
- countries: comes from I94_SAS_Labels_Descriptions.SAS
- visa: comes from I94_SAS_Labels_Descriptions.SAS
- I94 mode: comes from I94_SAS_Labels_Descriptions.SAS
- airport: comes from datahub.io,  it has airport information.

In [2]:
# Read in the data here
with open('I94_SAS_Labels_Descriptions.SAS') as labels_descriptions:
    raw_labels = labels_descriptions.read()

In [3]:
# extract country data
labels = raw_labels[raw_labels.index("I94CIT & I94RES - This format shows all the valid and invalid codes for processing"):]
labels = labels[:labels.index(';')]
lines = labels.splitlines()
fields = ['code', 'Country']
rows=[]
for i in range(2,len(lines)):
    rows.append(lines[i][1:].split("="))
with open('countries.csv', 'w') as f:
    write = csv.writer(f) 
    write.writerow(fields)
    write.writerows(rows)

In [6]:
# read city data
cities_df=spark.read.format("csv").option("header", "true").option("delimiter", ";").load("us-cities-demographics.csv")

In [7]:
cities_df.show()

+----------------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|            City|         State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race| Count|
+----------------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|   Silver Spring|      Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino| 25924|
|          Quincy| Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White| 58723|
|          Hoover|       Alabama|      38.5|      

In [8]:
# read airport data
airport_df=spark.read.format("csv").option("header", "true").option("delimiter", ",").load("airport-codes_csv.csv")

In [9]:
airport_df.show(3)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
+-----+-------------+--------------------+------------+---------+-----------+-----

In [10]:
# read sas_data
sas_df = spark.read.parquet("sas_data")

In [11]:
sas_df.show(3)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

In [12]:
# read country data
country_df=spark.read.format("csv").option("header", "True").option("delimiter", ",").load("countries.csv")

In [14]:
country_df.show(3)

+------+--------------------+
|  code|             Country|
+------+--------------------+
|  582 |  'MEXICO Air Sea...|
|  236 |       'AFGHANISTAN'|
|  101 |           'ALBANIA'|
+------+--------------------+
only showing top 3 rows



In [15]:
# read visa data
visa_df=spark.read.format("csv").option("header", "True").option("delimiter", "=").load("visa.csv")

In [16]:
visa_df.show()

+----+--------+
|code|VisaType|
+----+--------+
|   1|Business|
|   2|Pleasure|
|   3| Student|
+----+--------+



In [47]:
# read I94mode data
I94mode_df=spark.read.format("csv").option("header", "True").option("delimiter", ",").load("I94mode.csv")

In [48]:
I94mode_df.show()

+----+---------------+
|code|        I94mode|
+----+---------------+
|   1|          'Air'|
|   2|          'Sea'|
|   3|         'Land'|
|   9|'Not reported' |
+----+---------------+



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

#### these operations need to be done:

- city data may need "string or float" to "integer" format, also need to use sum aggregation to calculate the total population.

- airport data need to process iso_region column to get the state information

- sas_data need to change multiple columns data type to integer, also extract the arrive_date and split them into year, month and day


In [19]:
from pyspark.sql.types import *
cities_clean_df=cities_df.withColumn("Count",cities_df["Count"].cast('integer'))

In [20]:
from pyspark.sql.functions import sum as _sum
cities_clean_df = cities_clean_df.groupBy("City", "State", "Median Age", "Male Population",
                                     "Female Population" \
                                     , "Total Population", "Number of Veterans","Foreign-born",
                                     "Average Household Size", "State Code").agg(_sum("Count").alias('Total Count'))

In [21]:
cities_clean_df.show(3)

+----------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+-----------+
|      City|         State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|Total Count|
+----------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+-----------+
|Manchester| New Hampshire|      37.3|          54845|            55378|          110223|              5473|       14506|                   2.4|        NH|     123828|
| Charlotte|North Carolina|      34.3|         396646|           430475|          827121|             36046|      128897|                  2.52|        NC|     926239|
|    Skokie|      Illinois|      43.4|          31382|            33437|           64819|              1066|       27424|                  2.78|        IL|     

In [22]:
airport_clean_df=airport_df.where((airport_df["iso_country"]== "US")&airport_df['type'].isin(["large_airport", "medium_airport", "small_airport"]))

In [23]:
from pyspark.sql import SparkSession, SQLContext, GroupedData
from pyspark.sql.functions import *
airport_clean_df= airport_clean_df.withColumn("iso_region", substring(airport_df["iso_region"], 4, 2))

In [24]:
airport_clean_df.show(3)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|        KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|        AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|        AL|     Harvest|    00AL|     null|      00AL|-86.7703018188476...|
+-----+-------------+--------------------+------------+---------+-----------+-----

In [31]:
sas_clean_df=sas_df.withColumn("cicid", sas_df["cicid"].cast("integer")) \
                   .withColumn("i94yr", sas_df["i94yr"].cast("integer")) \
                   .withColumn("i94mon", sas_df["i94mon"].cast("integer")) \
                   .withColumn("i94mode", sas_df["i94mode"].cast("integer")) \
                   .withColumn("i94visa", sas_df["i94visa"].cast("integer")) \
                   .withColumn("biryear", sas_df["biryear"].cast("integer")) \
                   .withColumn("i94bir", sas_df["i94bir"].cast("integer")) \
                   .withColumn("i94res", sas_df["i94res"].cast("integer"))

In [32]:
sas_clean_df=sas_clean_df.withColumn("time_start", to_date(lit("01/01/1960"), "MM/dd/yyyy")) \
            .withColumn("arrive_date", expr("date_add(time_start, arrdate)")) \
            .withColumn("leave_date", expr("date_add(time_start, depdate)")) \

In [33]:
sas_clean_df=sas_clean_df.drop("time_start", "arrdate", "depdate")

In [34]:
sas_clean_df=sas_clean_df.withColumn("arrive_ymd", split(col("arrive_date"), "-")) \
            .withColumn("arrive_year", col("arrive_ymd")[0]) \
            .withColumn("arrive_month", col("arrive_ymd")[1]) \
            .withColumn("arrive_day", col("arrive_ymd")[2]).drop("arrive_ymd")

In [35]:
sas_clean_df.show(3)

+-------+-----+------+------+------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+-----------+----------+-----------+------------+----------+
|  cicid|i94yr|i94mon|i94cit|i94res|i94port|i94mode|i94addr|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|arrive_date|leave_date|arrive_year|arrive_month|arrive_day|
+-------+-----+------+------+------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+-----------+----------+-----------+------------+----------+
|5748517| 2016|     4| 245.0|   438|    LOS|      1|     CA|    40|      1|  1.0|20160430|     SYD| null|      G|      O|   null|      M|   1976|10292016|     F|  null|     QF|9.495387003E10|00011|     

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

Tranform data:

Transform demographics dataset grouping by state an calculate all the totals.
Transform inmigration dataset on order to get arrival date in different columns (year, month, day) for partitioning the dataset.

Generate Model (Star Schema):

Create all dimensions in parquet.
Create fact table in parquet particioned by year, month, day of th arrival date.
Insert in fact table only items with dimension keys right. For integrity and consistency.

#### Conceptual/Mapping
- We use Star Schema for this data
- There is another file named Data_dictionary.md show the fact table and dimension table content

#### fact table: sas_fact
#### dimension tables: airport, cities, country, I94mode, visa

- Transform the cities data: groupby by state and get the totals for total population, male and demale polulation.
- Transform the sas_data by join with other dimension tables.

In [36]:
# city data check the all the totals group by state 
cities_final_df = cities_clean_df.groupBy(cities_clean_df["State Code"].alias("State_code"), cities_clean_df["State"]) \
                  .agg({"Total Population": "sum", "Male Population": "sum", "Female Population": "sum"})

In [37]:
cities_final_df=cities_final_df.withColumnRenamed("sum(Male Population)","sum_Male_Population")
cities_final_df=cities_final_df.withColumnRenamed("sum(Total Population)","sum_Total_Population")
cities_final_df=cities_final_df.withColumnRenamed("sum(Female Population)","sum_Female_Population")

In [38]:
cities_final_df.show(3)

+----------+--------------+-------------------+--------------------+---------------------+
|State_code|         State|sum_Male_Population|sum_Total_Population|sum_Female_Population|
+----------+--------------+-------------------+--------------------+---------------------+
|        MT|       Montana|            87707.0|            181294.0|              93587.0|
|        NC|North Carolina|          1466105.0|           3060199.0|            1594094.0|
|        MD|      Maryland|           627951.0|           1312129.0|             684178.0|
+----------+--------------+-------------------+--------------------+---------------------+
only showing top 3 rows



In [49]:
sas_fact_df = sas_clean_df \
            .join(cities_final_df, sas_clean_df["i94addr"] == cities_final_df["State_Code"], "leftsemi") \
            .join(airport_clean_df, sas_clean_df["i94port"] == airport_clean_df["local_code"], "leftsemi") \
            .join(country_df, sas_clean_df["i94cit"] == country_df["code"], "leftsemi") \
            .join(visa_df, sas_clean_df["i94visa"] == visa_df["code"], "leftsemi") \
            .join(I94mode_df, sas_clean_df["i94mode"] == I94mode_df["code"], "leftsemi")

In [50]:
sas_fact_df.show(3)

+-------+-----+------+------+------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+-----------+----------+-----------+------------+----------+
|  cicid|i94yr|i94mon|i94cit|i94res|i94port|i94mode|i94addr|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|arrive_date|leave_date|arrive_year|arrive_month|arrive_day|
+-------+-----+------+------+------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+-----------+----------+-----------+------------+----------+
|5748522| 2016|     4| 245.0|   464|    HHW|      1|     HI|    57|      2|  1.0|20160430|     ACK| null|      G|      O|   null|      M|   1959|10292016|     M|  null|     NZ|9.498180283E10|00010|     

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

- save all the dataframe into parquet format, the fact table is saved with partion by year, month and day
- read all the parquet tables and check the row number to make sure it is not empty

In [51]:
# save each dataframe as parquet format

# first save these dimension Tables
cities_final_df.write.mode('overwrite').parquet("cities.parquet")
airport_clean_df.write.mode('overwrite').parquet("airport.parquet")
country_df.write.mode('overwrite').parquet("country.parquet")
visa_df.write.mode('overwrite').parquet("visa.parquet")
I94mode_df.write.mode('overwrite').parquet("I94mode.parquet")

In [52]:
sas_fact_df.write.partitionBy("arrive_year", "arrive_month", "arrive_day").mode('overwrite').parquet("sas_fact.parquet")

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [53]:
# Perform quality checks here
# read all the parquet files and check the row number'
cities_read_df=spark.read.parquet("cities.parquet")
airport_read_df=spark.read.parquet("airport.parquet")
country_read_df=spark.read.parquet("country.parquet")
visa_read_df=spark.read.parquet("visa.parquet")
I94mode_read_df=spark.read.parquet("I94mode.parquet")
sas_fact_read_df=spark.read.parquet("sas_fact.parquet")

In [54]:
cities_read_df.count()>0

True

In [55]:
airport_read_df.count()>0

True

In [110]:
country_read_df.count()>0

True

In [56]:
visa_read_df.count()>0

True

In [57]:
I94mode_read_df.count()>0

True

In [58]:
sas_fact_read_df.count()>0

True

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

- This project I use Spark to deal with the data and use parquet format to save the data, both of them can deal with large size of data. The data should be updated every day since each day we have new data

- The data was increased by 100x, I think the current method still works.

- The data populates a dashboard that must be updated on a daily basis by 7am every day, then we should use airflow to auto it.

- The database needed to be accessed by 100+ people, the we may use Redshift, this AWS data warehouse can shupport 500 connections